# Starbucks Capstone Challenge
## Data Cleaning

In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Datasets
The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**offers (portfolio.json)**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**customers (profile.json)**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**events (transcript.json)**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [2]:
# read in the json files
offers = pd.read_json('raw_data/portfolio.json', orient='records', lines=True)
customers = pd.read_json('raw_data/profile.json', orient='records', lines=True)
events = pd.read_json('raw_data/transcript.json', orient='records', lines=True)

# change column names and print df shape
offers.columns = ['reward','channels','min_spend','expire_days','offer_type','id']
customers.columns = ['gender','age','id','became_member_on','income']
events.columns = ['customer_id','event','value','hour']
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

offers has 10 rows
customers has 17000 rows
events has 306534 rows


### Remove 6 customers from all datasets who didn't receive offers

In [3]:
customer_funnel = events.groupby(['customer_id','event']).event.count().to_frame().rename({'event':'count'},axis=1).reset_index()
customer_funnel = pd.pivot(customer_funnel, index='customer_id', columns='event', values='count')
deleted_customers = customer_funnel[customer_funnel['offer received'].isnull()].index

events = events[~events.customer_id.isin(deleted_customers)]
customers = customers[~customers.id.isin(deleted_customers)]
events.reset_index(inplace=True, drop=True)
customers.reset_index(inplace=True, drop=True)

print(f'Removed {len(deleted_customers)} customers from all datasets who did not get any offers')
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

Removed 6 customers from all datasets who did not get any offers
offers has 10 rows
customers has 16994 rows
events has 306514 rows


### Clean the events data

In [4]:
# Clean the events data
for dic in events.value:
    if list(dic.keys()) == ['offer id']:
        dic['offer_id'] = dic.pop('offer id')

# Concat and transform the events data
temp = pd.DataFrame(events.value.to_list())
events = pd.concat([events,temp],axis=1)
events.rename(columns={'reward':'reward_received', 'amount':'amount_spent'},inplace=True)
events.drop('value',axis=1,inplace=True)
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')
events.head()

offers has 10 rows
customers has 16994 rows
events has 306514 rows


,customer_id,event,hour,offer_id,amount_spent,reward_received
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


### Create new feature in offers to capture most features in the data
- Format: offer_type + min_spend + expire_days + reward + channels

In [5]:
offers.channels = offers.channels.apply(lambda x: '_'.join(x))
offers['offer'] = offers.offer_type + '_' + offers.min_spend.astype(str) + '_' + offers.expire_days.astype(str) + '_' + offers.reward.astype(str) + '_' + offers.channels
offers.drop(['reward','min_spend','expire_days','channels'],axis=1,inplace=True)
offers

,offer_type,id,offer
0,bogo,ae264e3637204a6fb9bb56bc8210ddfd,bogo_10_7_10_email_mobile_social
1,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo_10_5_10_web_email_mobile_social
2,informational,3f207df678b143eea3cee63160fa8bed,informational_0_4_0_web_email_mobile
3,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo_5_7_5_web_email_mobile
4,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount_20_10_5_web_email
5,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,discount_7_7_3_web_email_mobile_social
6,discount,fafdcd668e3743c1bb461111dcafc2a4,discount_10_10_2_web_email_mobile_social
7,informational,5a8bc65990b245e5a138643cd4eb9837,informational_0_3_0_email_mobile_social
8,bogo,f19421c1d4aa40978ebb69ca19b0e20d,bogo_5_5_5_web_email_mobile_social
9,discount,2906b810c7d4411798c6938adc9daaa5,discount_10_7_2_web_email_mobile


### Clean customers data
- Change anomoly in age to null - when age = 118
- Create new feature in customers - membership_days

In [6]:
customers.age = customers.age.replace({118:np.nan})

# Create new feature for customer data - membership_days
# customers.became_member_on = pd.to_datetime(customers.became_member_on, format='%Y%m%d')
# most_recent = max(customers.became_member_on)
# customers['membership_days'] = customers.became_member_on.apply(lambda x: most_recent - x).dt.days
# customers['year_of_membership'] = customers.became_member_on.dt.year
# customers.drop('became_member_on',axis=1,inplace=True)

print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

offers has 10 rows
customers has 16994 rows
events has 306514 rows


### Export cleaned data into the cleaned_data folder

In [7]:
offers.to_csv('cleaned_data/offers.csv',index=False)
customers.to_csv('cleaned_data/customers.csv',index=False)
events.to_csv('cleaned_data/events.csv',index=False)